In [1]:
import numpy as np
import pandas as pd
import datetime as dt

from datetime import datetime, date
from pandas.tseries.offsets import CustomBusinessDay

In [2]:
# é spaghetti code, não analisei o código a fundo
def infer_calendar(dates):
    """
    Infer a calendar as pandas DateOffset from a list of dates.
    Parameters
    ----------
    dates : array-like (1-dimensional) or pd.DatetimeIndex
        The dates you want to build a calendar from
    Returns
    -------
    calendar : pd.DateOffset (CustomBusinessDay)
    """
    dates = pd.DatetimeIndex(dates)

    traded_weekdays = []
    holidays = []

    days_of_the_week = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
    for day, day_str in enumerate(days_of_the_week):

        weekday_mask = (dates.dayofweek == day)

        # keep only days of the week that are present
        if not weekday_mask.any():
            continue
        traded_weekdays.append(day_str)

        # look for holidays
        used_weekdays = dates[weekday_mask].normalize()
        all_weekdays = pd.date_range(dates.min(), dates.max(),
                                     freq=CustomBusinessDay(weekmask=day_str)
                                     ).normalize()
        _holidays = all_weekdays.difference(used_weekdays)
        _holidays = [timestamp.date() for timestamp in _holidays]
        holidays.extend(_holidays)

    traded_weekdays = ' '.join(traded_weekdays)
    return CustomBusinessDay(weekmask=traded_weekdays, holidays=holidays)

In [4]:
# Carrega Closing Price Diário - cpd
cpd_ati = pd.read_excel('../../data/economatica_1/fechamento_diario_ativas.xlsx',
                            skiprows = [1,2],
                            header=1,
                            index_col = 0,
                            na_values='-')
cpd_can = pd.read_excel('../../data/economatica_1/fechamento_diario_canceladas.xlsx',
                            skiprows = [1,2],
                            header=1,
                            index_col = 0,
                            na_values='-')
cpd = pd.concat([cpd_ati,cpd_can],axis=1,sort=False)
cpd.columns = cpd.columns.str.extract('(.*)$')[0]
cpd.columns.name = "ticker"
cpd.dropna(how='all',inplace=True) # cpd possui muitas linhas com NaN
cpd.dropna(axis=1, how='all',inplace=True) # cpd possui muitas colunas com NaN

In [28]:
# Calcula Closing Price Mensal - cpm
dates = cpd.index
calendar = infer_calendar(dates)
cme_cal = pd.offsets.CustomBusinessMonthEnd(calendar = calendar.calendar)
idm = pd.date_range(dates[0], dates[-1], freq=cme_cal)
cpm = cpd.loc[idm]
cpm

ticker,ABCB4,EALT3,EALT4,ADHM3,TIET3,TIET4,TIET11,AFLT3,BRGE3,BRGE5,...,WMBY4,WET4,WHMT3,WISA3,WISA4,ILMD3,ILMD4,ESTC4,ESTC11,ZIVI4
1986-01-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.007049e-12,NaN,8.338750e-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1986-02-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7.022509e-12,NaN,1.286146e-09,NaN,NaN,NaN,2.916550e-08,NaN,NaN,NaN
1986-03-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.580065e-11,NaN,3.180032e-09,NaN,NaN,NaN,8.749650e-08,NaN,NaN,NaN
1986-04-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.580065e-11,NaN,4.240042e-09,NaN,NaN,NaN,2.770722e-07,NaN,NaN,NaN
1986-05-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.860965e-11,NaN,4.055026e-09,NaN,NaN,NaN,2.221638e-07,NaN,NaN,NaN
1986-06-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.987370e-11,NaN,3.400022e-09,NaN,NaN,NaN,1.698899e-07,NaN,NaN,NaN
1986-07-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.176978e-11,NaN,3.700024e-09,NaN,NaN,NaN,1.568215e-07,NaN,NaN,NaN
1986-08-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.544952e-11,NaN,2.900019e-09,NaN,NaN,NaN,1.306846e-07,NaN,NaN,NaN
1986-09-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,8.497236e-12,NaN,2.147156e-09,NaN,NaN,NaN,9.213262e-08,NaN,NaN,NaN
1986-10-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.109557e-11,NaN,2.351648e-09,NaN,NaN,NaN,1.012805e-07,NaN,NaN,NaN


In [25]:
# Custom Month End: último dia do mês, usando um calendário customizado
# O último dia do mês é tranquilo de conseguir pois tem função pronta. O último dia do trimestre já é mais complicado...
# tem BQuarterEnd mas não tem CQuarterEnd
cme_cal = pd.offsets.CustomBusinessMonthEnd(calendar = calendar.calendar)
idm = pd.date_range(dates[0], dates[-1], freq=cme_cal)
cpm = cpd.loc[idm]
cpm
#cpd.loc['2018-01']

ticker,ABCB4,EALT3,EALT4,ADHM3,TIET3,TIET4,TIET11,AFLT3,BRGE3,BRGE5,...,WMBY4,WET4,WHMT3,WISA3,WISA4,ILMD3,ILMD4,ESTC4,ESTC11,ZIVI4
1986-01-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.007049e-12,NaN,8.338750e-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1986-02-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7.022509e-12,NaN,1.286146e-09,NaN,NaN,NaN,2.916550e-08,NaN,NaN,NaN
1986-03-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.580065e-11,NaN,3.180032e-09,NaN,NaN,NaN,8.749650e-08,NaN,NaN,NaN
1986-04-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.580065e-11,NaN,4.240042e-09,NaN,NaN,NaN,2.770722e-07,NaN,NaN,NaN
1986-05-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.860965e-11,NaN,4.055026e-09,NaN,NaN,NaN,2.221638e-07,NaN,NaN,NaN
1986-06-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.987370e-11,NaN,3.400022e-09,NaN,NaN,NaN,1.698899e-07,NaN,NaN,NaN
1986-07-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.176978e-11,NaN,3.700024e-09,NaN,NaN,NaN,1.568215e-07,NaN,NaN,NaN
1986-08-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.544952e-11,NaN,2.900019e-09,NaN,NaN,NaN,1.306846e-07,NaN,NaN,NaN
1986-09-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,8.497236e-12,NaN,2.147156e-09,NaN,NaN,NaN,9.213262e-08,NaN,NaN,NaN
1986-10-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.109557e-11,NaN,2.351648e-09,NaN,NaN,NaN,1.012805e-07,NaN,NaN,NaN
